In [1]:
import sys
sys.path.append('/workspace/cropClassification')
sys.path.append('/workspace/cropClassification/model')
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import torch
import pandas as pd
from torch.utils.data import DataLoader

from unet_uncertain import UNetWithUncertainty, UNetWithFiLM, UNetWithAttention, UNetWithAttentionDeep
from unet import originalUNet
from compiler import ModelCompiler
from dataloader import RoadsideCropImageDataset
from loss import AleatoricLoss, BalancedCrossEntropyLoss, BalancedCrossEntropyUncertaintyLoss #Picking between different loss function

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use GPU with CUDA
    print("Using CUDA")
elif torch.backends.mps.is_available():
    device = torch.device("mps")  # Use Apple M1/M2 GPU with MPS (Metal Performance Shaders)
    print("Using MPS")
else:
    device = torch.device("cpu")  # Fall back to CPU
    print("Using CPU")

### Configuration

In [3]:
config = {
    "model": {
        "type": "UNetWithUncertainty", 
        "params": {
            "in_channels": 10,  # Since we are using 9-channel input images
            "out_channels": 3   # Number of output classes for segmentation
        }
    },
    "training": {
        "epochs": 100,
        "batch_size": 32,
        "dropout_rate": 0.1,
        "learning_rate": 0.01,
        "optimizer": {
            "type": "Adam",
            "params": {
                "lr": 0.1
            }
        },
        "scheduler": {
            "type": "StepLR",
            "params": {
                "step_size": 10,
                "gamma": 0.8
            }
        },
        "criterion": BalancedCrossEntropyLoss,
        "classwise_weights": [0.14335682, 0.35709336, 0.49954982],
    },
    "validation": {
        "epochs": 50,
        "batch_size": 16
    },
    "dataset": {
        "train_csv": "/workspace/data/masked_data_csiss/training/train_chipping_csv_w_anc.csv",  # Path to the training DataFrame (includes npy file paths)
        "val_csv": "/workspace/data/masked_data_csiss/validation/validation_chipping_csv_w_anc.csv", # Path to the validation DataFrame (includes npy file paths)
        "train_root_path": "/workspace/data/masked_data_csiss/training",
        "val_root_path": "/workspace/data/masked_data_csiss/validation",
        "image_column": "img_chip_path",             # Column containing the image paths (npy files)
        "mask_column": "lbl_chip_path",              # Column containing the mask paths
        "train_mean": [121.65670372, 161.46653486, 158.23006705, 161.48324796, 227.81869621, 
                       146.17250505,  74.17987694,  98.37069385, 175.44668774, 0.59970105],  # Mean values for training set normalization
        "train_std": [52.67447251,  55.39850822,  78.60574882,  53.93343019, 59.63657872, 
                      109.67295329,  29.33554407,  55.42471356, 62.5700217 , 0.21684658],   # Std values for training set normalization
        "val_mean": [116.55829252, 159.9192156 , 157.07218772, 159.64730144, 232.93981733, 
                     140.17248727,  73.87612301, 104.39620349, 175.07042173, 0.59085459],    # Mean values for validation set normalization
        "val_std": [52.10727386,  54.40206462,  79.55377193,  53.06491964, 48.64101637, 
                    109.12006317,  28.95246699,  56.29244445, 63.20585391, 0.21176251],     # Std values for validation set normalization
        "classwise_norm": {
            "Other": {
                "mean": [110.14499, 138.28326, 117.67161, 138.71075, 233.10298, 95.24617, 
                         61.51174, 95.255585, 143.12497, 0.5121758],
                "std": [49.80741, 58.08042, 73.33471, 56.73531, 46.3197, 107.53641, 
                        27.30666, 48.58509, 61.8592, 0.22473]
            },
            "Maize": {
                "mean": [90.51625, 108.20322, 72.41352, 109.907, 203.1153, 24.78379, 
                         45.99272, 118.35542, 112.66669, 0.39319476],
                "std": [53.35697, 45.1693, 41.97862, 46.84034, 85.41561, 37.93971, 
                        21.92598, 52.69628, 46.29754, 0.1780796]
            },
            "Soybean": {
                "mean": [83.55291, 101.03232, 66.89771, 102.10976, 220.03397, 28.55189, 
                         47.94096, 116.89587, 103.48843, 0.36566228],
                "std": [49.91921, 47.52045, 41.91468, 48.74042, 67.89334, 47.91879, 
                        20.7751, 53.59672, 48.15541, 0.18223667]
            },
        }
    },
    "evaluation": {
        "filename": "csiss_street_view_crop_classification.csv",  # Evaluation metrics to be used
        "class_mapping": {
            0: "Background",
            1: "Maize",
            2: "Soybean"
        }
    }
}

### Loading Datasets

In [4]:
train_df = pd.read_csv(config['dataset']['train_csv'])
val_df = pd.read_csv(config['dataset']['val_csv'])

In [5]:
train_dataset = RoadsideCropImageDataset(
    dataframe=train_df,
    root_dir=config['dataset']['train_root_path'],
    usage='train',
    mean=config['dataset']['train_mean'], 
    std=config['dataset']['train_std'],  
    # classwise_norm=config['dataset']['classwise_norm'], 
    use_ancillary=True
)

val_dataset = RoadsideCropImageDataset(
    dataframe=val_df,
    root_dir=config['dataset']['val_root_path'],
    usage='val',
    mean=config['dataset']['val_mean'], 
    std=config['dataset']['val_std'],  
    # classwise_norm=config['dataset']['classwise_norm'], 
    use_ancillary=True
)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=config['training']['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['validation']['batch_size'], shuffle=False)

### Construct Model and Training

##### With Uncertainty (WIP)

In [ ]:
model = UNetWithUncertainty(n_channels=config['model']['params']['in_channels'],
                          n_classes=config['model']['params']['out_channels'],
                          ancillary_data_dim=3)
model_comp = ModelCompiler(model=model,
                           params_init=None)

In [ ]:
model_comp.fit(
    trainDataset=train_loader,
    valDataset=val_loader,
    epochs=config['training']['epochs'],
    optimizer_name=config['training']['optimizer']['type'],
    lr_init=config['training']['learning_rate'],
    lr_policy='steplr',  # Use 'steplr' as specified in config
    criterion=config['training']['criterion'],
    log=True,  # Enable logging for TensorBoard
    return_loss=False,
    use_ancillary=True,
    use_dropout=False,
    **config['training']['scheduler']['params'])

In [ ]:
model = UNetWithUncertainty(n_channels=config['model']['params']['in_channels'],
                          n_classes=config['model']['params']['out_channels'],
                          ancillary_data_dim=3)
model_comp = ModelCompiler(model=model,
                           params_init="/workspace/notebook/outputs/UNetWithUncertainty_ep50/chkpt/50_checkpoint.pth.tar")

In [ ]:
model_comp.evaluate(dataloader=val_loader,
                    num_classes=config['model']['params']['out_channels'],
                    out_name=config['evaluation']['filename'],  
                    class_mapping=config['evaluation']['class_mapping'],
                    log_uncertainty=True)

In [ ]:
model_comp.predict_image("/workspace/data/data/all_sv_imgs/IMG_2022_004.jpg", )

##### With only Crop Calendar (WIP)

In [ ]:
model = UNetWithFiLM(n_channels=config['model']['params']['in_channels'],
                     n_classes=config['model']['params']['out_channels'],
                     ancillary_data_dim=3,
                     use_dropout=False)
model_comp = ModelCompiler(model=model,
                           params_init=None)

In [ ]:
model_comp.fit(
    trainDataset=train_loader,
    valDataset=val_loader,
    epochs=config['training']['epochs'],
    optimizer_name=config['training']['optimizer']['type'],
    lr_init=config['training']['learning_rate'],
    lr_policy='steplr',  # Use 'steplr' as specified in config
    criterion=config['training']['criterion'],
    # class_weights=config['training']['classwise_weights'],
    log=True,  # Enable logging for TensorBoard
    return_loss=False,
    use_ancillary=True,
    **config['training']['scheduler']['params'])

In [ ]:
model_comp.evaluate(dataloader=val_loader,
                    num_classes=config['model']['params']['out_channels'],
                    class_mapping=config['evaluation']['class_mapping'],
                    out_name=config['evaluation']['filename'],  
                    log_uncertainty=True)

##### With Attention and Multiple FiLM

In [ ]:
model = UNetWithAttention(n_channels=config['model']['params']['in_channels'],
                     n_classes=config['model']['params']['out_channels'],
                     ancillary_data_dim=3,
                     dropout_rate=config['training']['dropout_rate'])
model_comp = ModelCompiler(model=model,
                           params_init=None)

In [ ]:
model_comp.fit(
    trainDataset=train_loader,
    valDataset=val_loader,
    epochs=config['training']['epochs'],
    optimizer_name=config['training']['optimizer']['type'],
    lr_init=config['training']['learning_rate'],
    lr_policy='steplr',  # Use 'steplr' as specified in config
    criterion=config['training']['criterion'],
    class_weights=config['training']['classwise_weights'],
    log=True,  # Enable logging for TensorBoard
    return_loss=False,
    use_ancillary=True,
    **config['training']['scheduler']['params'])

In [ ]:
# model = UNetWithAttention(n_channels=config['model']['params']['in_channels'],
#                           n_classes=config['model']['params']['out_channels'],
#                           ancillary_data_dim=3)
# model_comp = ModelCompiler(model=model,
#                            params_init="/workspace/notebook/outputs-UWA-ep50-lr0.01-batch32/UNetWithAttention_ep50/chkpt/final_checkpoint.pth.tar")

In [ ]:
model_comp.evaluate(dataloader=val_loader,
                    num_classes=config['model']['params']['out_channels'],
                    class_mapping=config['evaluation']['class_mapping'],
                    out_name=config['evaluation']['filename'],  
                    log_uncertainty=True)

In [ ]:

mask = model_comp.simple_predict(image_path="/workspace/data/all_sv_imgs/IMG_2022_279.jpg",
                                 csv_path="/workspace/data/masked_data_csiss/validation/validation_chipping_csv_w_anc.csv",
                                 save_path="/workspace/data/inference_result/")